<a href="https://colab.research.google.com/github/gan1014/Machine_learning-projects/blob/main/LBW_cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
pip install opencv-python numpy streamlit matplotlib pyngrok streamlit


In [25]:
!streamlit --version
!ngrok version


Streamlit, version 1.53.1
ngrok version 3.35.0
pyngrok version 7.5.0


In [6]:
import cv2
import numpy as np

def detect_ball(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Cricket ball color range (reddish)
    lower = np.array([0, 120, 70])
    upper = np.array([10, 255, 255])

    mask = cv2.inRange(hsv, lower, upper)
    mask = cv2.GaussianBlur(mask, (5, 5), 0)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        c = max(contours, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)

        if radius > 5:
            return int(x), int(y)

    return None


In [7]:
def detect_pad(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # White pad detection
    _, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)

    return thresh


In [8]:
import cv2
import numpy as np

def detect_stumps(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)

    lines = cv2.HoughLinesP(
        edges,
        1,
        np.pi / 180,
        threshold=100,
        minLineLength=100,
        maxLineGap=10
    )

    stump_positions = []

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            if abs(x1 - x2) < 10:  # vertical line
                stump_positions.append(x1)

    return stump_positions


In [9]:
import numpy as np

def lbw_decision(ball, pad_mask, stump_x):
    if ball is None or not stump_x:
        return "NOT OUT"

    x, y = ball

    # Check if ball hits pad
    if pad_mask[y, x] == 255:
        pad_hit = True
    else:
        pad_hit = False

    # Ball must be in line with stumps
    if pad_hit and min(stump_x) <= x <= max(stump_x):
        return "LBW OUT"

    return "NOT OUT"


In [11]:
import streamlit as st
import cv2
# from ball_detection import detect_ball
# from pad_detection import detect_pad
# from stump_detection import detect_stumps
# from lbw_logic import lbw_decision

st.title("🏏 Image-Based LBW Detection System")

uploaded_file = st.file_uploader("Upload Cricket Image", type=["jpg", "png","webp"])

if uploaded_file:
    image = cv2.imdecode(
        np.frombuffer(uploaded_file.read(), np.uint8),
        cv2.IMREAD_COLOR
    )

    ball = detect_ball(image)
    pad_mask = detect_pad(image)
    stumps = detect_stumps(image)

    decision = lbw_decision(ball, pad_mask, stumps)

    if ball:
        cv2.circle(image, ball, 6, (0, 255, 0), -1)

    for x in stumps:
        cv2.line(image, (x, 0), (x, image.shape[0]), (255, 0, 0), 2)

    st.image(image, channels="BGR")
    st.success(f"Decision: {decision}")

2026-01-30 14:44:20.485 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-30 14:44:20.487 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-30 14:44:20.488 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-30 14:44:20.489 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-30 14:44:20.490 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-30 14:44:20.491 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-30 14:44:20.491 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-30 14:44:20.492 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [28]:
%%writefile app.py
import streamlit as st
import cv2
import numpy as np

def detect_ball(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Cricket ball color range (reddish)
    lower = np.array([0, 120, 70])
    upper = np.array([10, 255, 255])

    mask = cv2.inRange(hsv, lower, upper)
    mask = cv2.GaussianBlur(mask, (5, 5), 0)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        c = max(contours, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)

        if radius > 5:
            return int(x), int(y)

    return None

def detect_pad(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # White pad detection
    _, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)

    return thresh

def detect_stumps(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)

    lines = cv2.HoughLinesP(
        edges,
        1,
        np.pi / 180,
        threshold=100,
        minLineLength=100,
        maxLineGap=10
    )

    stump_positions = []

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            if abs(x1 - x2) < 10:  # vertical line
                stump_positions.append(x1)

    return stump_positions

def lbw_decision(ball, pad_mask, stump_x):
    if ball is None or not stump_x:
        return "NOT OUT"

    x, y = ball

    # Check if ball hits pad
    if pad_mask[y, x] == 255:
        pad_hit = True
    else:
        pad_hit = False

    # Ball must be in line with stumps
    if pad_hit and min(stump_x) <= x <= max(stump_x):
        return "LBW OUT"

    return "NOT OUT"

st.title("🏏 Image-Based LBW Detection System")

uploaded_file = st.file_uploader("Upload Cricket Image", type=["jpg", "png","webp"])

if uploaded_file:
    image = cv2.imdecode(
        np.frombuffer(uploaded_file.read(), np.uint8),
        cv2.IMREAD_COLOR
    )

    ball = detect_ball(image)
    pad_mask = detect_pad(image)
    stumps = detect_stumps(image)

    decision = lbw_decision(ball, pad_mask, stumps)

    if ball:
        cv2.circle(image, ball, 6, (0, 255, 0), -1)

    for x in stumps:
        cv2.line(image, (x, 0), (x, image.shape[0]), (255, 0, 0), 2)

    st.image(image, channels="BGR")
    st.success(f"Decision: {decision}")

Overwriting app.py


In [29]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.28.0.12:8502
  External URL: http://34.171.168.60:8502

  Stopping...
  Stopping...


In [31]:
from pyngrok import ngrok
from google.colab import userdata

# Terminate any existing ngrok tunnels
ngrok.kill()

# Get the ngrok authentication token from Colab secrets
NGROK_AUTH_TOKEN = userdata.get("token")

# Authenticate ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start a ngrok tunnel for Streamlit on port 8501
public_url = ngrok.connect(8501)
print(f"Streamlit App URL: {public_url}")

# Run the Streamlit app in the background
!streamlit run app.py &

Streamlit App URL: NgrokTunnel: "https://reportable-lillianna-seamiest.ngrok-free.dev" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.171.168.60:8501

  Stopping...


In [27]:
from pyngrok import ngrok

# Replace "YOUR_AUTHTOKEN" with your actual ngrok authentication token
ngrok.set_auth_token("YOUR_AUTHTOKEN")
!streamlit run app.py &>/content/logs.txt &
public_url = ngrok.connect(8501)
print("Your Streamlit app is live at:", public_url)

ERROR:pyngrok.process.ngrok:t=2026-01-30T14:53:26+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour authtoken: token\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2026-01-30T14:53:26+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour authtoken: token\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour authtoken: token\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.

In [34]:
%%writefile app.py
import cv2
import numpy as np
import streamlit as st

st.set_page_config(page_title="LBW Video Detection", layout="centered")
st.title("🏏 LBW Detection System (Video-Based)")

uploaded_video = st.file_uploader(
    "Upload a Cricket Delivery Video",
    type=["mp4", "avi", "mov"]
)

# ---------- BALL DETECTION ----------
def detect_ball(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lower = np.array([0, 120, 70])
    upper = np.array([10, 255, 255])

    mask = cv2.inRange(hsv, lower, upper)
    mask = cv2.GaussianBlur(mask, (5, 5), 0)

    contours, _ = cv2.findContours(
        mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    if contours:
        c = max(contours, key=cv2.contourArea)
        ((x, y), r) = cv2.minEnclosingCircle(c)
        if r > 5:
            return int(x), int(y)
    return None

# ---------- PAD DETECTION ----------
def detect_pad(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
    return mask

# ---------- STUMP DETECTION ----------
def detect_stumps(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)

    lines = cv2.HoughLinesP(
        edges, 1, np.pi / 180,
        threshold=120,
        minLineLength=100,
        maxLineGap=10
    )

    stump_x = []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            if abs(x1 - x2) < 10:
                stump_x.append(x1)

    return stump_x

# ---------- LBW LOGIC ----------
def lbw_decision(ball_points, pad_mask, stumps):
    if len(ball_points) < 3 or not stumps:
        return "NOT OUT"

    # Last known ball position
    x, y = ball_points[-1]

    # Check pad hit
    if pad_mask[y, x] != 255:
        return "NOT OUT"

    # Predict ball path
    ys = [p[1] for p in ball_points]
    xs = [p[0] for p in ball_points]
    line = np.polyfit(ys, xs, 1)

    predicted_x = np.polyval(line, max(ys) + 50)

    if min(stumps) <= predicted_x <= max(stumps):
        return "LBW OUT"

    return "NOT OUT"

# ---------- MAIN ----------
if uploaded_video:
    tfile = open("temp.mp4", "wb","jpeg")
    tfile.write(uploaded_video.read())
    tfile.close()

    cap = cv2.VideoCapture("temp.mp4")
    stframe = st.empty()

    ball_points = []
    decision = "PROCESSING..."

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        ball = detect_ball(frame)
        pad_mask = detect_pad(frame)
        stumps = detect_stumps(frame)

        if ball:
            ball_points.append(ball)
            cv2.circle(frame, ball, 5, (0, 255, 0), -1)

        for x in stumps:
            cv2.line(frame, (x, 0), (x, frame.shape[0]), (255, 0, 0), 2)

        stframe.image(frame, channels="BGR")

    cap.release()

    decision = lbw_decision(ball_points, pad_mask, stumps)
    st.success(f"🏏 Decision: {decision}")


Overwriting app.py


In [35]:
from pyngrok import ngrok
from google.colab import userdata

# Terminate any existing ngrok tunnels
ngrok.kill()

# Get the ngrok authentication token from Colab secrets
NGROK_AUTH_TOKEN = userdata.get("token")

# Authenticate ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start a ngrok tunnel for Streamlit on port 8501
public_url = ngrok.connect(8501)
print(f"Streamlit App URL: {public_url}")

# Run the Streamlit app in the background
!streamlit run app.py &

Streamlit App URL: NgrokTunnel: "https://reportable-lillianna-seamiest.ngrok-free.dev" -> "http://localhost:8501"





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.171.168.60:8501

  Stopping...
